In [127]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
import pickle
import os
from datetime import datetime, date, timedelta

import selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
offers_db = pickle.load(open('db/offers_db', 'rb'))

In [3]:
offers_db

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
0,justjoin.it,junior,Software Developer Intern,AI Clearing,Warszawa,Not specified,6000.0,5000.0,7000.0,"[SQL, Python, Git]",https://justjoin.it/offers/ai-clearing-softwar...,2024-02-15 11:41,mazowieckie
1,justjoin.it,junior,AI/ML Engineer,MindPal,Warszawa,Praca zdalna,8500.0,7000.0,10000.0,"[AI, Machine Learning, C++]",https://justjoin.it/offers/mindpal-ai-ml-engin...,2024-02-15 11:41,mazowieckie
2,justjoin.it,junior,Junior Data Management Specialist,DXC Technology,Warszawa,Praca zdalna,NaN,NaN,NaN,"[data analytics, SQL, Python]",https://justjoin.it/offers/dxc-technology-juni...,2024-02-15 11:41,mazowieckie
3,justjoin.it,junior,Junior Python / DevOps Engineer,digatus,Warszawa,Praca zdalna,11000.0,9000.0,13000.0,"[Docker, DevOps, Python]",https://justjoin.it/offers/digatus-junior-pyth...,2024-02-15 11:41,mazowieckie
4,justjoin.it,junior,"Software Engineer, Early Career",Affirm,Warszawa,Not specified,16450.0,16450.0,16450.0,"[JavaScript, Kotlin, C++]",https://justjoin.it/offers/affirm-software-eng...,2024-02-15 11:41,mazowieckie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,pracuj.pl,mid,Specjalista ds. Narzędzi i Analiz BI,PKO Bank Polski SA,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[Python, SQL]",https://www.pracuj.pl/praca/specjalista-ds-nar...,2024-02-28 10:03,mazowieckie
1642,pracuj.pl,junior,Junior Data Scientist,A.T. Kearney Sp. z o.o.,Warszawa,Praca hybrydowa,10750.0,9500.0,12000.0,"[SQL, Python]",https://www.pracuj.pl/praca/junior-data-scient...,2024-02-28 10:03,mazowieckie
1643,pracuj.pl,junior,Młodszy Analityk Danych,Hart Sp. z o.o.,Wrocław,Praca zdalna,NaN,NaN,NaN,"[SQL, Oracle]",https://www.pracuj.pl/praca/mlodszy-analityk-d...,2024-02-28 10:03,dolnośląskie
1644,pracuj.pl,mid,Analityk IT / Feature Engineer,DomData AG Sp. z o.o.,Katowice,Praca zdalna,NaN,NaN,NaN,"[UML, BPMN, SQL, SOAP]",https://www.pracuj.pl/praca/analityk-it-featur...,2024-02-28 10:03,śląskie


In [237]:
# przeniesione
current_dir = os.getcwd()
driver_path = os.path.join(current_dir, 'app/chromedriver.exe')
service = Service(executable_path=driver_path)
chrome_options = Options()
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-notifications")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_window_size(2048, 1536)

url='https://it.pracuj.pl/praca?pn=2&its=big-data-science%2Cbusiness-analytics%2Cai-ml&itth=37%2C36'
driver.get(url)

In [238]:
try:
    WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.popup_p1c6glb0")))
    close_button = driver.find_element(By.CSS_SELECTOR, "div.popup_p1c6glb0")
    close_button.click()
except Exception as e:
    print("Exception while closing popup: ", e)
    
try:
    WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button[data-test='button-submitCookie']")))
    accept_button = driver.find_element(By.CSS_SELECTOR, "button[data-test='button-submitCookie']")
    accept_button.click()
except Exception as e:
    print("Nie udało się kliknąć przycisku 'Akceptuj wszystkie'", e)

In [239]:
elements1 = driver.find_elements(By.CSS_SELECTOR, "h5.tiles_r1rl4c7t.size-caption.core_t1rst47b[data-test='text-region']")

In [240]:
elements2 = driver.find_elements(By.CSS_SELECTOR, "h5.tiles_ttlhhld.size-caption.core_t1rst47b[data-test='text-region']")

In [241]:
elements = elements1 + elements2

In [242]:
actions = ActionChains(driver)

for element in elements:
    localization = element.text
    try:
        if localization and 'lokalizac' in localization:
            print(localization)
            actions.move_to_element(element).click().perform()
    except Exception as e:
        print(f"Nie udało się kliknąć przycisku {localization}, ponieważ: \n", e)
        

4 lokalizacje
2 lokalizacje
Miejsce pracy:Cała Polska (praca zdalna)
Siedziba firmy:12 lokalizacji
Miejsce pracy:Cała Polska (praca zdalna)
Siedziba firmy:2 lokalizacje


In [177]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [178]:
i=0
for offer in soup.find_all('div', class_="be8lukl core_po9665q"):
    whole_offer = offer.find('div', class_="c1fljezf")
    offer_details = whole_offer.find('div', class_="c1wygkax")
    location = offer_details.h5.text
    print(f'{i}: {location}')
    i+=1

0: Warszawa
1: Kraków, Dębniki
2: Warszawa
3: Kraków, Dębniki
4: Katowice
5: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa
6: Dąbrowa Górnicza
7: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Mokotów
8: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Mokotów
9: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Praga-Południe
10: Gdańsk, Oliwa
11: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Śródmieście
12: Poznań, Jeżyce
13: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa
14: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Częstochowa, Śródmieście
15: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Toruń, Wrzosy
16: Warszawa, Włochy
17: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Rzeszów
18: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Rzeszów
19: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Rzeszów
20: Miejsce pracy:Cała Polska (praca zd

In [229]:
links = []
offers = []

offer = soup.find_all('div', class_="be8lukl core_po9665q")[46]

whole_offer = offer.find('div', class_="c1fljezf")
offer_details = whole_offer.find('div', class_="c1wygkax")
keywords = whole_offer.find_all('div', class_='b1fdzgc4')

simple_location = offer_details.h5.text

if 'lokalizacj' in simple_location:
    locations = whole_offer.find_all('div', class_="tiles_lov4ye4")
    multi_locations = []
    
    for loc in locations:
        single_location = loc.a.text
        multi_locations.append(single_location)
        
    link = locations[0].a['href']
    location = multi_locations
    
else:
    link = offer_details.a['href']
    location = simple_location

if link in links:

    return None, links

links.append(link)

experience = offer_details.find('li').text
name = offer_details.h2.text
company = offer_details.h4.text
work_mode = offer_details.find_all('li')[-1].text
try:
    salary = offer_details.find('span', class_="s1jki39v").text.replace(u'\xa0', u'')
except (IndexError, AttributeError):
    salary = 'Undisclosed Salary'
techs = [keyword.text for keyword in keywords[0].find_all('span')] if keywords else []

new_offer = [experience, name, company, location, work_mode, salary, techs, link]

simple location: 16 lokalizacji
IF
Białystok
Bydgoszcz
Gdańsk
Katowice
Kielce
Kraków
Lublin
Łódź
Olsztyn
Opole
Poznań
Rzeszów
Szczecin
Warszawa
Wrocław
Zielona Góra
https://www.pracuj.pl/praca/senior-java-developer-bialystok,oferta,1003181326?s=1f7c2c91&searchId=MTcwOTIwMTcwNzkzOC4wNjQ=
['Białystok', 'Bydgoszcz', 'Gdańsk', 'Katowice', 'Kielce', 'Kraków', 'Lublin', 'Łódź', 'Olsztyn', 'Opole', 'Poznań', 'Rzeszów', 'Szczecin', 'Warszawa', 'Wrocław', 'Zielona Góra']


In [230]:
new_offer

['Starszy specjalista (Senior)',
 'Senior Java Developer',
 'Boldare',
 ['Białystok',
  'Bydgoszcz',
  'Gdańsk',
  'Katowice',
  'Kielce',
  'Kraków',
  'Lublin',
  'Łódź',
  'Olsztyn',
  'Opole',
  'Poznań',
  'Rzeszów',
  'Szczecin',
  'Warszawa',
  'Wrocław',
  'Zielona Góra'],
 'Praca hybrydowa',
 '16700–30300zł/ mies. (zal. od umowy)',
 ['Java',
  'Spring Boot',
  'Hibernate',
  'AWS',
  'PostgreSQL',
  'MS SQL',
  'Maven',
  'Gradle',
  'JUnit',
  'Mockito',
  'Docker',
  'Git'],
 'https://www.pracuj.pl/praca/senior-java-developer-bialystok,oferta,1003181326?s=1f7c2c91&searchId=MTcwOTIwMTcwNzkzOC4wNjQ=']

In [206]:
offer = soup.find_all('div', class_="be8lukl core_po9665q")[46]
whole_offer = offer.find('div', class_="c1fljezf")
offer_details = whole_offer.find('div', class_="c1wygkax")
keywords = whole_offer.find_all('div', class_='b1fdzgc4')
link = offer_details.a['href']
company = offer_details.h4.text
location = offer_details.h5.text
name = offer_details.h2.text

In [207]:
location

'16 lokalizacji'

In [208]:
localizations = whole_offer.find_all('div', class_="tiles_lov4ye4")

In [214]:
localizations[0].a['href']

'https://www.pracuj.pl/praca/senior-java-developer-bialystok,oferta,1003181326?s=1f7c2c91&searchId=MTcwOTIwMTcwNzkzOC4wNjQ='

In [213]:
localizations[0].a.text

'Białystok'

In [ ]:
<div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-bialystok,oferta,1003181326?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Białystok</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div>

In [191]:
<div class="tiles_l17prsi5"><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-bialystok,oferta,1003181326?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Białystok</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-bydgoszcz,oferta,1003181324?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Bydgoszcz</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-gdansk,oferta,1003181322?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Gdańsk</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-katowice,oferta,1003181320?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Katowice</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-kielce,oferta,1003181300?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Kielce</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-krakow,oferta,1003181301?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Kraków</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-lublin,oferta,1003181304?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Lublin</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-lodz,oferta,1003181307?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Łódź</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-olsztyn,oferta,1003181309?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Olsztyn</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-opole,oferta,1003181310?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Opole</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-poznan,oferta,1003181311?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Poznań</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-rzeszow,oferta,1003181312?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Rzeszów</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-szczecin,oferta,1003181313?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Szczecin</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-warszawa,oferta,1003181314?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Warszawa</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-wroclaw,oferta,1003181316?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Wrocław</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div><div class="tiles_lov4ye4"><a href="https://www.pracuj.pl/praca/senior-java-developer-zielona-gora,oferta,1003181318?s=1f7c2c91&amp;searchId=MTcwOTIwMTcwNzkzOC4wNjQ=" data-test="link-offer" class="core_btsqgu core_n194fgoq">Zielona Góra</a><span class="tiles_s84op4y core_c1s7lqrd"><span aria-hidden="true" class="core_ig18o8w size-medium position-center"><svg viewBox="0 0 20 20" fill="none"><path d="M9.605 1.271a.419.419 0 0 1 .783 0l2.12 6.008h5.984a.418.418 0 0 1 .272.737l-5 4.146 2.092 6.287a.419.419 0 0 1-.644.47l-5.216-3.827-5.219 3.827a.419.419 0 0 1-.642-.47l2.093-6.287-5.001-4.146a.419.419 0 0 1 .272-.737h5.985l2.12-6.008z" stroke="#676B8D" stroke-width="1.6" stroke-linecap="round" stroke-linejoin="round"></path></svg></span></span></div></div>

'https://pracodawcy.pracuj.pl/company/13894778'